In [ ]:
# from supabase import create_sync_client

from core.database.supabase_client import get_supabase_client

supabase = get_supabase_client()

In [23]:
def get_video_status(
    video_id: str
):
    status_response =  supabase.table("videos").select("processed").eq("video_id",video_id).execute()
    print(status_response)
    if status_response and len(status_response.data) > 0:
        if status_response.data[0]["processed"] == "SUCCESS":
            return {"status":"processed"}
    else:
        return {"status":"failed"}
status = get_video_status("fWO6Ibrt8UE")


INFO:httpx:HTTP Request: GET https://sofowbahdzuboflvuprw.supabase.co/rest/v1/videos?select=processed&video_id=eq.fWO6Ibrt8UE "HTTP/2 200 OK"


data=[{'processed': 'SUCCESS'}] count=None


In [2]:
from yt_rag.llm_service.gemini_client import get_gemini_client


llm_client = get_gemini_client()

In [59]:
from google.genai.types import Part


image = Part.from_uri(file_uri=uploaded_file.uri, mime_type="image/jpeg")

In [60]:
image

Part(
  file_data=FileData(
    file_uri='https://generativelanguage.googleapis.com/v1beta/files/cos48ojqv1zm',
    mime_type='image/jpeg'
  )
)

In [74]:
# from io import BytesIO
# from google.genai.types import ContentListUnionDict
# from PIL import Image



url = "https://sofowbahdzuboflvuprw.supabase.co/storage/v1/object/public/images/kids/jaivik_smile.jpg?"

from google.genai.types import ContentListUnionDict, UploadFileConfig
UploadFileConfig
url = "https://sofowbahdzuboflvuprw.supabase.co/storage/v1/object/public/images/kids/jaivik_smile.jpg?"

history: ContentListUnionDict = [
    {
        "role": "user",
        "parts": [
            {"text": "Describe the content in this image"},
            {"inline_data": {"mime_type": "image/jpeg", "data": data.content}}
        ],
    }, "How r u doing my name is yogesh, and can u see the image"
] 

In [76]:
response = llm_client.models.generate_content(model="gemini-2.0-flash", contents=history)
print(response.text)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:google_genai.models:AFC remote call 1 is done.


I'm doing well, Yogesh, thank you for asking! Yes, I can see the image you provided. It features a baby lying on their stomach, smiling widely. The baby is wearing a bright blue t-shirt and has a dark mark on their cheek and forehead. They're lying on a white textured cloth, and there are flowers and greenery around them. In the background, there's a refrigerator, a window with bars, and part of a room.



In [ ]:
-- corresponding sql query
SELECT chat.id, chat_messages.*
FROM chat
LEFT JOIN chat_messages chat.id = chat_messages.id
WHERE chat.video_id = "zdeXctev" AND chat.user_id = "FVatzG-ADGEAsrgre"
ORDER BY chat_messages.created_at ASC;

In [3]:
# Gemini RAG tool call
import json
import requests
from yt_rag.vector_store.embeddings import create_text_embeddings
from yt_rag.vector_store.pg_vector_operations import PG_Vector_search

def get_relevent_multimodal_data(video_id: str, prompt: str):
    query_embedding = create_text_embeddings(prompt)
    vector_search = PG_Vector_search()
    
    frames = vector_search.match_frames(video_id, query_embedding)
    transcripts = vector_search.match_transcript(video_id, query_embedding)
    print(frames, transcripts)
    as_messages = {
        "role": "user",
        "parts": [
            {"text": "These are the relevant multimodal (Contains youtube frames and text transcript chunks) data to user's question. Use this data to answer intelligently if helpfull"}
        ]
    }
    for frame in frames:
        data = requests.get(frame["frame_url"][1:])
        image_bytes = data.content
        as_messages["parts"].append(
            {"inline_data":{ "mime_type":"image/jpeg", "data": image_bytes }}
        )
    data = ""
    for transcript in transcripts:
        json_data = json.loads(transcript["content"])
        data += json_data["page_content"]
    as_messages["parts"].append(
        {"text": data}
    )
    
    return as_messages
        
    
    
# messages = get_relevent_multimodal_data("wCNIhFdhgLE", "who is powell")

In [ ]:
class LLMClient:
    def __init__(self, video_id: str, user_id: str, chat_id: str):
        self.history = []
        self.llm_client = get_gemini_client()
        self.supabase = get_supabase_client()
        self.video_id = video_id
        self.user_id = user_id
        self.chat_id = chat_id
        self.tools = []
        
    def load_history(self):
        response = (
            supabase.table("chat")
            .select("*", "chat_messages(*, 'order=created_at.asc')")
            .eq("user_id", self.user_id)
            .eq("video_id", self.video_id)
            .execute()
        )
        if response.data:
            messages = response.data[0]["chat_messages"]
            history = []
            for msg in messages:
                construct_msg = {}
                construct_msg["role"] = "model" if msg["role"] == "assistant" else "user"
                construct_msg["parts"] = [msg["content"]]
                history.append(history)
            
            self.history = history
        else:
            self.history = []
    
    def run(self, prompt):
        self.load_history()
        
        contents = self.history.append({
            "role": "user",
            "parts": prompt
        })
        
        while True:
        
            response = llm_client.models.generate_content(
                model="gemini-2.0-flash",
                content = contents,
                config = {
                    "system_instruction": "Answer honestly everytime",
                    "temperature": 0.5,
                    "tools": [tools]
                },                
            )
            if response.function_calls:
                for function_call in response.function_calls:
                    if function_call.name == "get_relevent_multimodal_data":
                        relevant_content = get_relevent_multimodal_data(
                            video_id= function_call.args["video_id"],
                            query= function_call.args["query"]
                        )

            
            
            
            supabase.table("chat_messages").insert({
                "role":"ASSISTANT",
                "content":response.text,
                "chat_id":self.chat_id
            }).execute()
            
            return response.text
            

SyntaxError: invalid syntax (1686419161.py, line 9)

In [133]:
from pydantic import Field, BaseModel
from typing import Dict

tools_dict = [
    {
        "name": "get_relevent_multimodal_data",
        "description": "get relevent frames and transcript chunks to users query from youtube video",
        "parameters": {
            "type": "object",
            "properties": {
                "video_id" : {"type": "string", "description": "Unique identifier of a youtube video"},
                "query": {"type": "string", "description": "efficient Query for the rag to tool to find relvent data"}
            },
            "required": ["video_id", "query"]
        }
    }
]

class InputSchema(BaseModel):
    video_id: str = Field(description="Unique identifier of a youtube video")
    query: str = Field(description="efficient Query for the rag to tool to find relvent data")

class Schema(BaseModel):
    name: str
    description: str
    input_schema: Dict[str, object]
    
tools = [
    Schema(
        name="get_relevent_multimodal_data",
        description="get relevent frames and transcript chunks to users query from youtube video",
        input_schema = InputSchema.model_json_schema()
    )
]

In [4]:
# tools=[
#     {
#         function_declarations=[
#             {
#                 name="get_relevent_multimodal_data",
#                 description="get relevent frames and transcript chunks to users query from youtube video",
#                 parameters={
#                     ""
#                 }
#             }
#         ]
#     }
# ]

from google.genai import types

fun_dec = types.FunctionDeclaration(
    name="get_relevent_multimodal_data",
    description="get relevent frames and transcript chunks to users query from youtube video",
    parameters=types.Schema(
        type=types.Type.OBJECT,
        properties={
            "video_id": types.Schema(type=types.Type.STRING, description="Unique identifier of a YouTube video"),
            "query": types.Schema(type=types.Type.STRING, description="Efficient query for the RAG tool to find relevant data"),
        },
        required=["video_id", "query"]
    ),
)

tool = types.Tool(function_declarations=[fun_dec])

In [7]:
response = llm_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[{
        "role":"user",
        "parts":[{"text": "What is jorem powell talking in this video? link to video=https://m.youtube.com/watch?v=wCNIhFdhgLE"}]
    }],
    config={
        "temperature":0.75,
        "system_instruction": "You are a youtube assistant assistant. You have access to multimodal rag tool \"get_relevent_multimodal_data\" which u can use to relevant frames and text chunks of the video. To use the tool u have to send video_id and query to the tool. video_id is the unique indentifier to a youtube video for example \"jU4d39Z_9GU\" is video_id for youtube url \"https://m.youtube.com/watch?v=jU4d39Z_9GU\"",
        "tools": [tool]
    }
)

In [10]:
# for function_call in response.function_calls:
#     print(get_relevent_multimodal_data(
#         video_id=function_call.args["video_id"],
#         prompt=function_call.args["query"]
#     ))
import requests
data_response = get_relevent_multimodal_data(
    video_id=response.function_calls[0].args["video_id"],
    prompt=response.function_calls[0].args["query"]
)

matching frames to url
Matching transcript to chunks
Matching transcript to chunks completed
[{'frame_embed_id': 'b054a727-816e-4b17-8eae-160296aa1298', 'frame_url': "'https://sofowbahdzuboflvuprw.supabase.co/storage/v1/object/public/images/./wCNIhFdhgLE/frame_0002.jpg", 'distance': 0.787158974321069, 'similarity': 0.212841025678931, 'created_at': '2025-09-18T04:24:44.878734+00:00'}, {'frame_embed_id': 'cf230a88-dd8e-4c9d-86fd-86e0972df306', 'frame_url': "'https://sofowbahdzuboflvuprw.supabase.co/storage/v1/object/public/images/./wCNIhFdhgLE/frame_0001.jpg", 'distance': 0.787495857478174, 'similarity': 0.212504142521826, 'created_at': '2025-09-18T04:24:44.878734+00:00'}] [{'transcript_embed_id': 'b30842fc-3bb3-416d-b076-808f37492eb0', 'content': '{"page_content":"In support of our goals.\\nAnd in light of the shift in the balance of risks.\\nToday, the Federal Open Market Committee decided to lower our policy interest\\nrate by a quarter percentage point. The labor market was in very s

In [11]:
data_response

{'role': 'user',
 'parts': [{'text': "These are the relevant multimodal (Contains youtube frames and text transcript chunks) data to user's question. Use this data to answer intelligently if helpfull"},
  {'inline_data': {'mime_type': 'image/jpeg',
    'data': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00\x10Lavc61.31.101\x00\xff\xdb\x00C\x00\x08\x08\x08\t\x08\t\x0b\x0b\x0b\x0b\x0b\x0b\r\x0c\r\r\r\r\r\r\r\r\r\r\r\x0e\x0e\x0e\x11\x11\x11\x0e\x0e\x0e\r\r\x0e\x0e\x10\x10\x11\x11\x12\x13\x12\x11\x11\x11\x11\x13\x13\x14\x14\x14\x18\x18\x17\x17\x1c\x1c\x1d"")\xff\xc4\x00\xc3\x00\x00\x00\x07\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x07\x04\x02\x03\x01\x08\x01\x00\x01\x05\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x05\x03\x01\x02\x06\x10\x00\x02\x01\x03\x02\x02\x07\x04\x06\x07\x04\x05\t\x05\x06\x07\x01\x02\x03\x04\x00\x11!\x121\x05"AaQq\x13\x06\x812\x91\x14#BR\xb1r\xa1\x15\xd1b\xe1\xc13\x82\x92\xb2\xa2\xf0s\x07ScC\xd2

In [13]:
contents = [
    data_response,
    {
        "role": "user",
        "parts": [{"text": "Explain the following content"}]
    }
]

contents

[{'role': 'user',
  'parts': [{'text': "These are the relevant multimodal (Contains youtube frames and text transcript chunks) data to user's question. Use this data to answer intelligently if helpfull"},
   {'inline_data': {'mime_type': 'image/jpeg',
     'data': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00\x10Lavc61.31.101\x00\xff\xdb\x00C\x00\x08\x08\x08\t\x08\t\x0b\x0b\x0b\x0b\x0b\x0b\r\x0c\r\r\r\r\r\r\r\r\r\r\r\x0e\x0e\x0e\x11\x11\x11\x0e\x0e\x0e\r\r\x0e\x0e\x10\x10\x11\x11\x12\x13\x12\x11\x11\x11\x11\x13\x13\x14\x14\x14\x18\x18\x17\x17\x1c\x1c\x1d"")\xff\xc4\x00\xc3\x00\x00\x00\x07\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x07\x04\x02\x03\x01\x08\x01\x00\x01\x05\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x05\x03\x01\x02\x06\x10\x00\x02\x01\x03\x02\x02\x07\x04\x06\x07\x04\x05\t\x05\x06\x07\x01\x02\x03\x04\x00\x11!\x121\x05"AaQq\x13\x06\x812\x91\x14#BR\xb1r\xa1\x15\xd1b\xe1\xc13\x82\x92\xb2\xa2\xf0s\x07ScC

In [180]:
contents[1]

{'role': 'user', 'parts': [{'text': 'Explain the following content'}]}

In [15]:
final_response = llm_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[data_response, {
        "role": "user",
        "parts": [{"text": "Explain the following content"}]
    }]
)

In [16]:
final_response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.355909538269043,
      content=Content(
        parts=[
          Part(
            text="""Jerome Powell, the Federal Reserve Chair, is speaking about the Federal Open Market Committee's decision to lower the policy interest rate by a quarter percentage point. He mentions that the labor market is in good condition with strong job creation. He refers to revised job creation numbers for May, June, July, and August, indicating a positive trend. He also notes the arrival of a new committee member and emphasizes the committee's unity in pursuing their dual mandate goals. He frames the interest rate cut as a risk management measure, with the expectation that inflation will rise, although possibly not as high as previously anticipated. He also notes that the impact of tariffs on inflation has been slower and smaller than expected.
"""
          ),
        ],
        role='mod